In [ ]:
import psi4
from qcportal import PortalClient
from pprint import pprint as pp
from qcelemental.models import Molecule

In [1]:
from setup_qcfractal import setup_qcarchive_qcfractal
import os

setup_qcarchive_qcfractal(
    QCF_BASE_FOLDER=os.path.join(os.getcwd(), "qcfractal"),
    start=False,
    reset=False,
    db_config={
        "name": None,
        "enable_security": "false",
        "allow_unauthenticated_read": None,
        "logfile": None,
        "loglevel": None,
        "service_frequency": 10,
        "max_active_services": None,
        "heartbeat_frequency": None,
        "log_access": None,
        "database": {
            "base_folder": None,
            "host": None,
            "port": 5432,
            "database_name": "qca",
            "username": None,
            "password": None,
            "own": None,
        },
        "api": {
            "host": None,
            "port": 7777,
            "secret_key": None,
            "jwt_secret_key": None,
        },
    },
)

/home/amwalla3/gits/cybershuttle_demo/qcfractal

--------------------------------------------------------------------------------
Python executable:  /home/amwalla3/miniconda3/envs/p4_qcml/bin/python
QCFractal version:  0.59
QCFractal alembic revision:  d5988aa750ae
pg_ctl path:  /home/amwalla3/miniconda3/envs/p4_qcml/bin/pg_ctl
PostgreSQL server version:  PostgreSQL 17.4 on x86_64-conda-linux-gnu, compiled by x86_64-conda-linux-gnu-cc (conda-forge gcc 13.3.0-2) 13.3.0, 64-bit
--------------------------------------------------------------------------------


Displaying QCFractal configuration below
--------------------------------------------------------------------------------
access_log_keep: 0
allow_unauthenticated_read: true
api:
  extra_flask_options: null
  extra_waitress_options: null
  host: localhost
  jwt_access_token_expires: 3600
  jwt_refresh_token_expires: 86400
  jwt_secret_key: 29oNF42M8_IMtQL7cW_A9RGqo7sUg5hLPqrhsVXGaGo
  num_threads_per_worker: 4
  port: 7777
  secret

In [ ]:
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start > qcfractal_server.log & disown

# NOTE kill server when finished by running:
#     ps aux | grep qcfractal-server | awk '{ print $2 }'
#     kill -9 <PID>

In [ ]:
!qcfractal-compute-manager --config=`pwd`/qcfractal/resources.yml > qcfractal_compute.log & disown
# NOTE kill server when finished by running:;
#    ps aux | grep qcfractal-server | awk '{ print $2 }'
#    kill -9 <PID>

In [2]:
# Running a single job
client = PortalClient("http://localhost:7777", verify=False)
for rec in client.query_records():
    pp(rec)

mol = Molecule.from_data(
    """
     0 1
     O  -1.551007  -0.114520   0.000000
     H  -1.934259   0.762503   0.000000
     H  -0.599677   0.040712   0.000000
     --
     0 1
     O   1.350625   0.111469   0.000000
     H   1.680398  -0.373741  -0.758561
     H   1.680398  -0.373741   0.758561

     units angstrom
     no_reorient
     symmetry c1
"""
)

psi4.set_options(
    {"basis": "aug-cc-pvdz", "scf_type": "df", "e_convergence": 6, "freeze_core": True}
)

client.add_singlepoints(
    [mol],
    "psi4",
    driver="energy",
    method="b3lyp",
    basis="aug-cc-pvdz",
    keywords={"scf_type": "df", "e_convergence": 6, "freeze_core": True},
    tag="local",
)

for rec in client.query_records():
    pp(rec.dict)
    pp(rec.error)

SinglepointRecord(id=1, record_type='singlepoint', is_service=False, properties={'pe energy': 0.0, 'scf dipole': [1.01443376121482, 0.030397167953621462, 4.7350182585600975e-12], 'calcinfo_nmo': 82, 'dft xc energy': -15.09393332583476, 'return_energy': -152.89658734330823, 'return_result': -152.89658734330823, 'scf_xc_energy': -15.09393332583476, 'calcinfo_natom': 6, 'calcinfo_nbeta': 10, 'current dipole': [1.01443376121482, 0.030397167953621462, 4.7350182585600975e-12], 'current energy': -152.89658734330823, 'return_hessian': None, 'scf_iterations': 8, 'calcinfo_nalpha': 10, 'calcinfo_nbasis': 82, 'dft vv10 energy': 0.0, 'return_gradient': None, 'dft total energy': -152.89658734330823, 'scf_total_energy': -152.89658734330823, 'scf_dipole_moment': [1.01443376121482, 0.030397167953621462, 4.7350182585600975e-12], 'scf_total_hessian': None, 'scf total energies': [-152.18329179022422, -152.54809853364748, -151.78477048281337, -152.8958414937564, -152.89589353340773, -152.8965718707634, -1

In [3]:
# Now create S22 Interaction Energy Dataset
from s22 import geoms

# geoms is a list of qcelemental Molecule objects that can be used to create a
# QCArchive dataset
print(geoms)

[Molecule(name='H4O2', formula='H4O2', hash='49ecd61'), Molecule(name='C2H4O4', formula='C2H4O4', hash='cb6a49d'), Molecule(name='C2H6N2O2', formula='C2H6N2O2', hash='b1ca3e5'), Molecule(name='C8H8N4O4', formula='C8H8N4O4', hash='13c83b3'), Molecule(name='C10H11N3O', formula='C10H11N3O', hash='3b8e723'), Molecule(name='C10H11N7O2', formula='C10H11N7O2', hash='3520f9b'), Molecule(name='C2H8', formula='C2H8', hash='67d7d29'), Molecule(name='C4H8', formula='C4H8', hash='47f28b2'), Molecule(name='C7H10', formula='C7H10', hash='4810149'), Molecule(name='C12H12', formula='C12H12', hash='420be36'), Molecule(name='C8H8N4', formula='C8H8N4', hash='3072d92'), Molecule(name='C8H8N4O4', formula='C8H8N4O4', hash='9f67f10'), Molecule(name='C14H13N', formula='C14H13N', hash='00ea319'), Molecule(name='C10H11N7O2', formula='C10H11N7O2', hash='4af63d5'), Molecule(name='C4H6', formula='C4H6', hash='2f8f067'), Molecule(name='C6H8O', formula='C6H8O', hash='22eeb90'), Molecule(name='C6H9N', formula='C6H9N',

In [4]:
# Create client dataset

ds_name = 'S22'

try:
    ds = client.add_dataset("singlepoint", ds_name,
                            f"Dataset to contain {system}")
    print(f"Added {system} as dataset")
except Exception:
    ds = client.get_dataset("singlepoint", system)
    print(f"Found {system} dataset, using this instead")
    print(ds)

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[51], line 6
      4     ds = client.add_dataset("singlepoint", "S22",
      5                             f"Dataset to contain S22 dimers")
----> 6     print(f"Added {system} as dataset")
      7 except Exception:

NameError: name 'system' is not defined

During handling of the above exception, another exception occurred:

NameError                                 Traceback (most recent call last)
Cell In[51], line 8
      6     print(f"Added {system} as dataset")
      7 except Exception:
----> 8     ds = client.get_dataset("singlepoint", system)
      9     print(f"Found {system} dataset, using this instead")
     10     print(ds)

NameError: name 'system' is not defined


In [ ]:
!ps aux | grep qcfractal-server | awk '{ print $2 }'